# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [16]:
!nvidia-smi

## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the links are dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace, or you can use [the Kaggle API](https://www.kaggle.com/general/74235) to directly download the data into colab.**


### Download train/test metadata

In [17]:
# Main link
!wget -O libriphone.zip "https://github.com/xraychen/shiny-robot/releases/download/v1.0/libriphone.zip"

# Backup Link 0
# !pip install --upgrade gdown
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
# !pip install --upgrade gdown
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Backup link 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# Backup link 3
# !wget -O libriphone.zip "https://www.dropbox.com/s/p2ljbtb2bam13in/libriphone.zip?dl=1"

!unzip -q libriphone.zip
!ls libriphone

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [1]:
import os
import gc
import random
import pandas as pd
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
      y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, y
    else:
      return X

def preprocess_seq_data(split, feat_dir, phone_path, train_ratio=0.8, train_val_seed=42):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

          # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    usage_list = [line.strip('\n') for line in usage_list]

    X, y = [], []
    for i, fname in enumerate(usage_list):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])
            y.append(label)
        X.append(feat)

    if mode != 'test':
        return X[:percent], X[percent:], y[:percent], y[percent:]
    else:
        return X
    

## Define Dataset

In [2]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence, pad_sequence

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

class SequenceLabelingDataset(Dataset):
    def __init__(self, X, y=None) -> None:
        self.data = X
        if y is not None:
            self.label = y
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

def pad_collate(batch):
    if isinstance(batch[0], tuple):
        xx, yy = zip(*batch)
    else:
        xx, yy = batch, None
    x_lens = [len(x) for x in xx]
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=-1)
    if yy:
        y_lens = [len(y) for y in yy]
        yy_pad = pad_sequence(yy, batch_first=True, padding_value=-1)
        return xx_pad, yy_pad, x_lens, y_lens
    return xx_pad, x_lens

## Define Model

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import GRU

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim, dropout=0):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            # nn.BatchNorm1d(output_dim),
            nn.LayerNorm(output_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = self.block(x)
        return x

class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

class Model(nn.Module):
    def __init__(self, input_dim, output_dim=41, rnn_layers=1, hidden_layers=1, hidden_dim=256, dropout=0):
        super().__init__()
        self.rnn = GRU(input_size=input_dim, hidden_size=hidden_dim, num_layers=rnn_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Sequential(
            BasicBlock(2*hidden_dim, hidden_dim, dropout=dropout),
            *[BasicBlock(hidden_dim, hidden_dim, dropout=dropout) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x, x_len):
        x = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        x, _ = self.rnn(x)
        x, _ = pad_packed_sequence(x, batch_first=True)
        x = self.fc(x)
        return x

In [4]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [5]:
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda:3


## Hyper-parameters

In [20]:
# data prarameters
concat_nframes = [31]              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.8               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 123                        # random seed
batch_size = 32                # batch size
num_epoch = 40                   # the number of training epoch
learning_rate = 1e-5          # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved
feat_dir = '/data/cjh/kaggle/libriphone/libriphone/feat/'
phone_path = '/data/cjh/kaggle/libriphone/libriphone/'

# model parameters
# input_dim = [39 * cn for cn in concat_nframes] # the input dim of the model, you should not change the value
input_dim = 39
hidden_layers = 1              # the number of hidden layers
hidden_dim = 512               # the hidden dim
rnn_layers = 2
dropout = 0.3

In [21]:
import sys
sys.stdout = open('log.txt', 'w')

## Sequence labeling model

### Prepare dataset and model

In [8]:
def unpadded(padded, length):
    assert padded.size(0) == len(length)
    ret = []
    for i, l in enumerate(length):
        ret.append(padded[i, :l])
    return torch.cat(ret, 0)

In [22]:
train_X, val_X, train_y, val_y = preprocess_seq_data(split='train', feat_dir=feat_dir, phone_path=phone_path, train_ratio=train_ratio)

train_set = SequenceLabelingDataset(train_X, train_y)
val_set = SequenceLabelingDataset(val_X, val_y)

del train_X, train_y, val_X, val_y
gc.collect()

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, collate_fn=pad_collate)


In [23]:
same_seeds(seed)

model = Model(input_dim=input_dim, hidden_dim=hidden_dim, rnn_layers=rnn_layers, hidden_layers=hidden_layers, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate*50)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, eta_min=learning_rate, T_max=num_epoch)


### Training

In [ ]:
best_acc = 0.0
for epoch in tqdm(range(num_epoch), desc='Epoch'):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    train_count = 0.0
    val_count = 0.0
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader, desc='Training', leave=False, position=0)):
        features, labels, x_len, y_len = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features, x_len) 
        
        outputs = unpadded(outputs, x_len)
        labels = unpadded(labels, y_len)
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_count += sum(x_len)
        train_loss += loss.item()

    scheduler.step()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader, desc='Validation', leave=False, position=0)):
                features, labels, x_len, y_len = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features, x_len)
                
                outputs = unpadded(outputs, x_len)
                labels = unpadded(labels, y_len)
                loss = criterion(outputs, labels)
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_count += sum(x_len)
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/train_count, train_loss/len(train_set), val_acc/val_count, val_loss/len(val_set)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/val_count))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/train_count, train_loss/len(train_set)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


## Testing

In [17]:
# del train_loader, val_loader
gc.collect()

# load data
test_X = preprocess_seq_data(split='test', feat_dir=feat_dir, phone_path=phone_path)
test_set = SequenceLabelingDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=pad_collate)

#load model
model = Model(input_dim=input_dim, hidden_dim=hidden_dim, rnn_layers=rnn_layers, hidden_layers=hidden_layers, dropout=dropout).to(device)
model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [18]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features, x_len = batch
        features = features.to(device)

        outputs = model(features, x_len)
        outputs = unpadded(outputs, x_len)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

100%|██████████| 34/34 [00:04<00:00,  8.09it/s]


## DNN - Prepare dataset and model

In [ ]:
import gc

# preprocess data
for cn in concat_nframes:
    train_X, train_y = preprocess_data(split='train', feat_dir=feat_dir, phone_path=phone_path, concat_nframes=cn, train_ratio=train_ratio)
    val_X, val_y = preprocess_data(split='val', feat_dir=feat_dir, phone_path=phone_path, concat_nframes=cn, train_ratio=train_ratio)

    # get dataset
    train_set = LibriDataset(train_X, train_y)
    val_set = LibriDataset(val_X, val_y)

    # remove raw feature to save memory
    del train_X, train_y, val_X, val_y
    gc.collect()

    # get dataloader
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    # fix random seed
    same_seeds(seed)

    # create model, define a loss function, and optimizer
    model = Classifier(input_dim=39*cn, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
    # model.load_state_dict(torch.load(model_path))
    criterion = nn.CrossEntropyLoss() 
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate*50)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, eta_min=learning_rate, T_max=epoch)
    best_acc = 0.0
    print(f'concat_nframes={cn}')
    for epoch in tqdm(range(num_epoch), desc='Epoch'):
        train_acc = 0.0
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0
        
        # training
        model.train() # set the model to training mode
        for i, batch in enumerate(tqdm(train_loader, desc='Training', leave=False, position=0)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() 
            outputs = model(features) 
            
            loss = criterion(outputs, labels)
            loss.backward() 
            optimizer.step() 
            
            _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
            train_acc += (train_pred.detach() == labels.detach()).sum().item()
            train_loss += loss.item()
        
        # validation
        if len(val_set) > 0:
            model.eval() # set the model to evaluation mode
            with torch.no_grad():
                for i, batch in enumerate(tqdm(val_loader, desc='Validation', leave=False, position=0)):
                    features, labels = batch
                    features = features.to(device)
                    labels = labels.to(device)
                    outputs = model(features)
                    
                    loss = criterion(outputs, labels) 
                    
                    _, val_pred = torch.max(outputs, 1) 
                    val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                    val_loss += loss.item()

                print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                    epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
                ))

                # if the model improves, save a checkpoint at this epoch
                if val_acc > best_acc:
                    best_acc = val_acc
                    torch.save(model.state_dict(), model_path)
                    print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
        else:
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
            ))

    # if not validating, save the last epoch
    if len(val_set) == 0:
        torch.save(model.state_dict(), model_path)
        print('saving model at last epoch')


In [ ]:
del train_loader, val_loader
gc.collect()

60

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# load data
test_X = preprocess_data(split='test', feat_dir=feat_dir, phone_path=phone_path, concat_nframes=concat_nframes[0])
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

1078it [00:02, 367.95it/s]


In [ ]:
# load model
model = Classifier(input_dim=input_dim[0], hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1263/1263 [00:03<00:00, 379.18it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [19]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))